In [1]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score

In [65]:
# Read in prepped dataset
titanic = pd.read_csv('../data/classification_dataset.csv')
titanic['Survived'] = titanic['Survived'].astype('category')

In [66]:
# Define variables
variables = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 
             'Fare', 'Embarked', 'ticket_count',
             'spouse_present', 'cabin_letter', 'cabin_number', 
             'ticket_letter', 'ticket_number', 'sibling_present']

# Define categorical variables for OneHotEncoder
categorical_variables = ['Sex', 'Embarked', 'cabin_letter', 'ticket_letter']

In [67]:
# Set up train test split
X = titanic[variables]
y = titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42, stratify=y)

In [68]:
pipe = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse=False, drop='first'), categorical_variables)
            ],
            remainder='passthrough')),
        ('pf', PolynomialFeatures(interaction_only=True, include_bias=False)),
        ('vt', VarianceThreshold()),
        ('scaler', StandardScaler()),
        ('logistic', LogisticRegression())
    ]
)

In [69]:
pipe.fit(X_train, y_train)

c:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['Sex', 'Embarked',
                                                   'cabin_letter',
                                                   'ticket_letter'])])),
                ('pf',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('vt', VarianceThreshold()), ('scaler', StandardScaler()),
                ('logistic', LogisticRegression())])

In [78]:
features = list(pipe['ct'].named_transformers_['ohe'].get_feature_names_out(categorical_variables))
features += [x for x in X_train.columns if x not in categorical_variables]
features = list(pipe['pf'].get_feature_names_out(features))
features = list(np.array(features)[pipe['vt'].get_support()])

survived = 1
idx = list(pipe['logistic'].classes_).index(survived)

coefficients = pd.DataFrame({
    'variable': ['intercept'] + features,
    'coefficient': [pipe['logistic'].intercept_[idx]] + list(pipe['logistic'].coef_[idx])
})

pipe['logistic'].intercept_[idx]
idx

IndexError: index 1 is out of bounds for axis 0 with size 1

In [75]:
titanic.dtypes

Unnamed: 0            int64
PassengerId           int64
Survived           category
Pclass                int64
Sex                  object
Age                 float64
SibSp                 int64
Parch                 int64
Fare                float64
Embarked             object
ticket_count          int64
spouse_present      float64
cabin_letter         object
cabin_number          int64
ticket_letter        object
ticket_number         int64
sibling_present     float64
dtype: object